In [ ]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.apps.icp4d-experiences-lite-xx.demo.ibmcloud.com",
    "username": "admin",
    "password": "passwordhere"
}

In [ ]:
DB_CREDENTIALS = {
    "jdbcurl": "jdbc:db2://icp4d-experiences-lite-xxworker-04.demo.ibmcloud.com:xxx/BLUDB",
    "hostname": "icp4d-experiences-lite-xxworker-04.demo.ibmcloud.com",
    "username": "user999",
    "password": "password",
    "port": 111,
    "db": "BLUDB"
}

SCHEMA_NAME = 'OPENSCALE'

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4 | tail -n 1

In [ ]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id'] = 'openshift'
WML_CREDENTIALS['version'] = '3.0.1'

In [ ]:
MODEL_NAME = 'Mortgage Default - OpenScale'
DEPLOYMENT_NAME = 'Mortgage Default - OpenScale'

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [ ]:
space_details = wml_client.spaces.list()

In [ ]:
space_name = "OpenScale Deployment Space"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["guid"]
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]
wml_client.set.default_space(space_id)

In [ ]:
wml_client.repository.list_models()

In [ ]:
deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/emartensibm/mortgage-default/master/Mortgage_Full_Records.csv'
df_raw = pd.read_csv(url)
df = df_raw.drop('ID', axis=1)
df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
categorical_features = ['AppliedOnline','Residence','Location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

scaled_features = ['Income','Yrs_at_Current_Address','Yrs_with_Current_Employer',\
                   'Number_of_Cards','Creditcard_Debt','Loan_Amount','SalePrice']
scale_transformer = Pipeline(steps=[('scale', MinMaxScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('scaler', scale_transformer, scaled_features)
    ]
)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

In [ ]:
X = df.drop('MortgageDefault', axis=1)
y = df['MortgageDefault']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

model = clf.fit(X_train, y_train)
res_predict = model.predict(X_test)
print("model score: %.3f" % clf.score(X_test, y_test))
print(classification_report(y_test, res_predict, target_names=["False", "True"]))

In [ ]:
deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
metadata = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: 'scikit-learn_0.22-py3.6',
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.22'
}

# Name the columns
cols=["Income","AppliedOnline","Residence","Yrs_at_Current_Address","Yrs_with_Current_Employer",\
      "Number_of_Cards","Creditcard_Debt","Loans","Loan_Amount","SalePrice","Location"]
      
saved_model = wml_client.repository.store_model(model=model, meta_props=metadata, training_data=X_train,\
                                                training_target=y_train, feature_names=cols,\
                                                label_column_names=["MortgageDefault"] )

saved_model

In [ ]:
model_uid = saved_model['metadata']['guid']
print("Deploying model", model_uid)

meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment = wml_client.deployments.create(artifact_uid=model_uid, meta_props=meta_props)
deployment_uid = wml_client.deployments.get_uid(deployment)

In [ ]:
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# OpenScale

In [ ]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [ ]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing datamart')
except:
    print('Setting up datamart')
    ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP())
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

In [ ]:
print(binding_uid)

In [ ]:
ai_client.data_mart.bindings.list_assets(binding_uid=binding_uid)

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [ ]:
cos_credentials = {
    "apikey": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
    "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
    "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
    "iam_url": 'https://iam.bluemix.net/oidc/token',
    "cos_hmac_keys": {
        "access_key_id": "2d1be760f19241d695a534960da6eb80",
        "secret_access_key": "e1252b952f47a6b3f42305b8ffe6f9bd7d10e45f966b9a62"
    },
    "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
    "iam_apikey_description": "Auto-generated for key 2d1be760-f192-41d6-95a5-34960da6eb80",
    "iam_apikey_name": "FastStartLab",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Reader",
    "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/7d8b3c34272c0980d973d3e40be9e9d2::serviceid:ServiceId-568ba191-a3bf-48f2-a30c-f3a4af7ec61d",
    "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::"
}

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='MortgageDefault',
    prediction_column='prediction',
    probability_column='probability',
    feature_columns = ['AppliedOnline','Residence','Location','Income','Yrs_at_Current_Address','Yrs_with_Current_Employer',\
                   'Number_of_Cards','Creditcard_Debt','Loan_Amount','Loans','SalePrice'],
    categorical_columns = ['AppliedOnline','Residence','Location'],
    training_data_reference = {
        'type': 'cos',
        'location': {
            'bucket': 'faststartlab-donotdelete-pr-nhfd4jnhlxgpc7',
            'file_name': 'Mortgage_Full_Records.csv',
            'firstlineheader': True,
            'file_format': 'csv'
        },
        'connection': cos_credentials,
        'name': 'training data reference'
    }
))

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

In [ ]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

In [ ]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)
data['fields'][1:]

In [ ]:
scoring_payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'][1:],
                           "values": [],
                           "meta": {
                               "fields": ["ID"],
                               "values": []
                           }
                       }
                    ]
                  }

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, 101):
    value_to_score = random.choice(data['values'])
    scoring_payload['input_data'][0]['values'].append(value_to_score[1:])
    scoring_payload['input_data'][0]['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])
print(len(scoring_payload['input_data'][0]['values']))

In [ ]:
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
print(predictions)

In [ ]:
time.sleep(10)
subscription.payload_logging.get_records_count()

In [ ]:
subscription.quality_monitoring.enable(threshold=0.8, min_records=50)

The cell below for fairness monitoring is commented out due to a configuration bug that breaks automated debias when configuring with the Python client.

In [ ]:
# subscription.fairness_monitoring.enable(
#     features=[
#         Feature("AppliedOnline", majority=['NO'], minority=['YES'], threshold=0.90)
#     ],
#     favourable_classes=['APPROVED'],
#     unfavourable_classes=['DENIED'],
#     min_records=100
# )

Instead, we'll use the REST client to configure fairness.

In [ ]:
import requests
import base64
def get_token():
    iam_url = WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    authorization = base64.b64encode(bytes('{0}:{1}'.format(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"]), 'utf-8')).decode("ascii")
    headers = {"Authorization": "Basic "+ authorization}
    get_token_resp = requests.get(iam_url, headers=headers, verify=False)
    get_token_resp = json.loads(get_token_resp.text)
    token = get_token_resp["accessToken"]
    return token
                                                                                                         
def get_headers():
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer " + get_token()
    return headers

In [ ]:
fairness_monitoring_url = WOS_CREDENTIALS["url"] + "/v1/fairness_monitoring"
payload = {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "asset_id": subscription.get_details()["metadata"]["guid"],
    "deployment_id": deployment_uid,
    "parameters": {
        "features": [{
            "feature": "AppliedOnline",
            "majority": ["NO"],
            "minority": ["YES"],
            "threshold": 0.90
        }],
        "favourable_class": ["APPROVED"],
        "unfavourable_class": ["DENIED"],
        "min_records": 100,
        "class_label": "prediction"
    }
}

requests.post(fairness_monitoring_url, json=payload, headers=get_headers(), verify=False)

In [ ]:
subscription.drift_monitoring.enable(threshold=0.05, min_records=100)

In [ ]:
from ibm_ai_openscale.supporting_classes import *

subscription.explainability.enable()

In [ ]:
transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'].values[0]

print(transaction_id)

In [ ]:
explain_run = subscription.explainability.run(transaction_id=transaction_id, background_mode=False, cem=False)

In [ ]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=100)

In [ ]:
feedback_set = df.values.tolist()

In [ ]:
feedback_payload = []
for _ in range(0, 250):
    feedback_payload.append(random.choice(feedback_set))
print(feedback_payload[0])

In [ ]:
subscription.feedback_logging.store(feedback_payload)